## makemore: becoming a backprop ninja

In [ ]:
# there no change change in the first several cells from last lecture

In [3]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [4]:
# download the names.txt file from github
!wget https://raw.githubusercontent.com/karpathy/makemore/master/names.txt

--2024-04-19 02:57:56--  https://raw.githubusercontent.com/karpathy/makemore/master/names.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228145 (223K) [text/plain]
Saving to: ‘names.txt’

names.txt           100%[===================>] 222.80K  --.-KB/s    in 0.01s   

2024-04-19 02:57:56 (22.9 MB/s) - ‘names.txt’ saved [228145/228145]



In [5]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [6]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [7]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
  X, Y = [], []

  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [ ]:
# ok biolerplate done, now we get to the action:

In [8]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [9]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

4137


In [10]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

In [11]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.3552, grad_fn=<NegBackward0>)

In [12]:
logits.shape, h.shape, W2.shape, b2.shape

(torch.Size([32, 27]),
 torch.Size([32, 64]),
 torch.Size([64, 27]),
 torch.Size([27]))

In [ ]:
probs.shape, counts.shape, counts_sum_inv.shape

(torch.Size([32, 27]), torch.Size([32, 27]), torch.Size([32, 1]))

In [ ]:
hpreact.shape, embcat.shape, W1.shape, b1.shape

(torch.Size([32, 64]),
 torch.Size([32, 30]),
 torch.Size([30, 64]),
 torch.Size([64]))

In [ ]:
bndiff.shape, hprebn.shape, bnmeani.shape

(torch.Size([32, 64]), torch.Size([32, 64]), torch.Size([1, 64]))

In [ ]:
bndiff.shape, bnvar_inv.shape, bnraw.shape, bndiff[1,1], bnvar_inv[0,1], bnraw[1,1]

(torch.Size([32, 64]),
 torch.Size([1, 64]),
 torch.Size([32, 64]),
 tensor(0.0533, grad_fn=<SelectBackward0>),
 tensor(0.8880, grad_fn=<SelectBackward0>),
 tensor(0.0474, grad_fn=<SelectBackward0>))

In [ ]:
print(logprobs.shape, loss, logprobs[range(n), Yb].shape)
print(logprobs[range(n), Yb]) #This goes through rows 0-31 where each row is the logits of that example in the batch and get the logit at [row, Yb[row]]
#loss = -(probability of each correct guess).mean() = -1/batch_size * (prob1+prob2+prob3+...+probbatch_size)
#So, the derivative of loss with respect to any probability (say prob1) is -1/n (n is batch_size)
#However, since only certain logits (the correct value ones) are used to calculate loss, the ones that aren't used should have a gradient of zero
#since any change to them will not impact the loss
"""
FULL SUMMARIZED EXPLANATION
We find the dlogprobs by finding the derivative of the loss function with respect to each value in log probs. logprobs is a tensor of dimension
(batch_size, vocab_size) where each value represents the probability of each next token for each example. For example log_probs[3,5] is the predicted
probability of the next token for example 4 being token 6 in our vocab. Our loss is determined by taking the mean prediction value of the correct
token for each batch. For example, say our correct tokens for a batch of 5 are [9, 7, 1, 0, 3], then our loss would be the mean of:
log_prob[0, 9], log_prob[1, 7], log_prob[2, 1], log_prob[3, 0], log_prob[4, 3]
and then we inverse the sign, since this would come out negative by default. So our loss in this example looks like:
-1/batch_size * (log_prob[0, 9] + log_prob[1, 7] + log_prob[2, 1] + log_prob[3, 0] + log_prob[4, 3])
so, each value of log prob that has an impact on our loss has a gradient of -1/batch_size. However, this isn't the end. As we see, not all the values
in log prob impact the loss. Only the value of the probability of the correct next prediction for each example in the batch has an impact. This means
only 1 in vocab_size values per example have any impact on the loss function and therefore any gradient. So, since these other predictions aren't
used to calculate the loss, their gradients are zero, so first, we'll create an array of the same dimensions as log_probs of all zeros:
dlogprobs = torch.zeros_like(logprobs)
then, we'll set the gradients of the values that do impact the loss function to their value of -1/batch_size
dlogprobs[range(n), Yb] = -1.0/n
"""
print(logits.max(1, keepdims=True).indices)

torch.Size([32, 27]) tensor(-3.3620, grad_fn=<MeanBackward0>) torch.Size([32])
tensor([-3.9692, -3.1061, -3.5329, -3.2271, -3.9774, -3.5791, -3.1945, -3.9710,
        -3.2862, -4.3547, -3.1859, -1.7402, -2.8166, -2.9472, -3.0727, -3.0888,
        -3.7725, -2.9578, -3.6136, -3.4615, -2.8770, -2.9503, -4.4031, -4.1514,
        -3.5050, -2.9804, -3.0967, -3.9578, -2.8524, -3.4913, -3.2362, -3.2286],
       grad_fn=<IndexBackward0>)
tensor([[ 1],
        [ 2],
        [19],
        [15],
        [15],
        [25],
        [16],
        [ 3],
        [21],
        [ 8],
        [15],
        [ 3],
        [22],
        [18],
        [ 7],
        [ 5],
        [ 2],
        [ 1],
        [22],
        [19],
        [15],
        [19],
        [22],
        [22],
        [23],
        [ 5],
        [22],
        [20],
        [24],
        [ 8],
        [23],
        [13]])


In [ ]:
# Exercise 1: backprop through the whole thing manually,
# backpropagating through exactly all of the variables
# as they are defined in the forward pass above, one by one

# -----------------
# YOUR CODE HERE :)
# -----------------
dlogprobs = torch.zeros_like(logprobs) # dL/dlogprobs = (d/dlogprobs)(loss) (See above cell for full explanation)
dlogprobs[range(n), Yb] = -1.0/n
dprobs = (1.0/probs) * dlogprobs #dL/dprobs = (dlogprobs/dprobs) * dlogprobs (find the derivative of logprobs with respect to each value of probs and multiply that by dlogprobs)
dcounts_sum_inv = (counts * dprobs).sum(1, keepdims=True) #derivative of probs with respect to counts_sum_inv * dprobs
dcounts = counts_sum_inv * dprobs # derivative of probs w/ respect to each value of counts * dprobs
dcounts_sum = -(counts_sum**-2) * dcounts_sum_inv
dcounts += torch.ones_like(counts) * dcounts_sum #This is what also causes the wierd sumation in dcounts_sum_inv. Full explanation below.
dnorm_logits = norm_logits.exp() * dcounts #since counts = norm_logits.exp(), the derivative of counts w/ respect to norm_logits is the same (x.exp() mean e^x and derivative of e^x is e^x)
dlogits = dnorm_logits.clone() #full explanation below
dlogit_maxes = (-1.0 * torch.ones_like(logit_maxes) * dnorm_logits).sum(1, keepdims=True) #full explanation below
dlogits_init = torch.zeros_like(logits) #Full explanation of these three rows below
dlogits_init[range(n), logits.max(1).indices] = 1.0
dlogits += dlogits_init * dlogit_maxes
dh = dlogits @ W2.T #Full explanation of dh, dW2, b2 below (dimensions need to work out)
dW2 = h.T @ dlogits
db2 = dlogits.sum(0)
dhpreact = (1.0 - h**2) * dh #full explanation below
dbngain = (bnraw * dhpreact).sum(0, keepdims=True) #Full exp. of these three below
dbnraw = bngain * dhpreact
dbnbias = dhpreact.sum(0, keepdims=True)
dbndiff = bnvar_inv * dbnraw
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdims=True)
dbnvar = (-0.5 * (bnvar + 1e-5)**-1.5) * dbnvar_inv
dbndiff2 = ((1/(n-1))*torch.ones_like(bndiff2)) * dbnvar
dbndiff += (bndiff*2) * dbndiff2
dhprebn = dbndiff.clone()
dbnmeani = (-dbndiff).sum(0)
dhprebn += 1.0/n * torch.ones_like(hprebn) * dbnmeani
dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0)
demb = dembcat.view(emb.shape)
dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
  for j in range(Xb.shape[1]):
    ix = Xb[k,j]
    dC[ix] += demb[k,j]

cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10
bngain          | exact: False | approximate: True  | maxdiff: 2.7939677238464355e-09
bnbias          | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09
bnraw  

# Explanation of Each Derivative

**We use the chain rule because this is how we get it to be the derivative of the final Loss with respect to the current variable**

## dlogprobs

**dlogprobs = dL/dlogprobs**

We find the dlogprobs by finding the derivative of the loss function with respect to each value in log probs. logprobs is a tensor of dimension
(batch_size, vocab_size) where each value represents the probability of each next token for each example. For example log_probs[3,5] is the predicted
probability of the next token for example 4 being token 6 in our vocab. Our loss is determined by taking the mean prediction value of the correct
token for each batch. For example, say our correct tokens for a batch of 5 are [9, 7, 1, 0, 3], then our loss would be the mean of:
log_prob[0, 9], log_prob[1, 7], log_prob[2, 1], log_prob[3, 0], log_prob[4, 3]
and then we inverse the sign, since this would come out negative by default. So our loss in this example looks like:
-1/batch_size * (log_prob[0, 9] + log_prob[1, 7] + log_prob[2, 1] + log_prob[3, 0] + log_prob[4, 3])
so, each value of log prob that has an impact on our loss has a gradient of -1/batch_size. However, this isn't the end. As we see, not all the values
in log prob impact the loss. Only the value of the probability of the correct next prediction for each example in the batch has an impact. This means
only 1 in vocab_size values per example have any impact on the loss function and therefore any gradient. So, since these other predictions aren't
used to calculate the loss, their gradients are zero, so first, we'll create an array of the same dimensions as log_probs of all zeros:
dlogprobs = torch.zeros_like(logprobs)
then, we'll set the gradients of the values that do impact the loss function to their value of -1/batch_size
dlogprobs[range(n), Yb] = -1.0/n

## dprobs

**dprobs = dL/dprobs = (dlogprobs/dprobs) * (dL/dlogprobs)**

Since we're finding dlogprobs/dprobs, we're finding d/dprobs(probs.log) which is equal to d/dprobs(ln(probs)) which comes out to 1/probs. We then mutliply by the derivative of logprobs because of the chain rule.

## dcounts_sum_inv

**dcounts_sum_inv = dL/dcounts_sum_inv = (dprobs/dcounts_sum_inv) * (dL/dprobs)**

Since probs = counts * counts_sum_inv and counts has a shape [batch_size, vocab_size] and counts_sum_inv has a shape [batch_size,1], each row in counts will be multiplied by a single value: the value of counts_sum_inv at that row. Example: Each value in counts[0] will be mutliplied by the value at counts_sum_inv[0,0]. This is why we take the sum of counts * probs by row. (We use counts because d/dx(a*x) = a and counts is a in this scenario.

## dcounts

**Part 1**

Since probs = counts * counts_sum_inv and counts has a shape [batch_size, vocab_size] and counts_sum_inv has a shape [batch_size,1], each row in counts will be multiplied by a single value: the value of counts_sum_inv at that row. Example: Each value in counts[0] will be mutliplied by the value at counts_sum_inv[0,0]. We multiply counts_sum_inv by dprobs because the equation is essentially counts_sum_inv[i](counts[i]) and we can rewrite this as a*x and so d/dx(a*x) = a and then we apply the chain rule. We don't need to sum because no value in counts is affecting the output value (probs) multiple times.

**Part 2**

Since counts_sum is just a summation, then dcounts_sum/dcounts is just going to be one, or an array of ones in this case that passes the gradients from above straight through.

Since counts is used in both counts_sum and probs, we have two functions that we need to take the derivative of with respect to counts and then we sum the gradients.

##dcounts_sum

Application of power rule, then chain rule.

##dnorm_logits


norm_logits.exp() = e^norm_logits and d/dx(e^x) = e^x
so the derivative of norm_logits is norm_logits.exp()

##dlogits

**Part 1**
Simple enough

**Part 2**

When finding dlogit_maxes/dlogits, only the max logits in each row have an impact on the loss and therefore only they should receive a gradient value. This is why we first create an array of zeros and then set the locations where a max value in to 1 and then multiply this mostly zeros array by the chain rule derivative so that only the max values have a gradient and all others' are zero.

##dh, dW2, dembcat, dW1

Finding the derivative for these involves a lot matrix operations and would take a lot of time by hand. You can do a simpler version to get your head around the operations and how the derivatives will look or you can do it this way: Look at the dimensions of the output of the matrix multiplication, the weights matrix, values matrix, and bias matrix for the neuron and then just make it so the dimensions work and know that one will need to be transposed (dimensions switched) in order for it to work properly.

**Full Example**



```
# Linear layer 2
logits = h @ W2 + b2
logits.shape = ([32, 27])
h.shape = ([32, 64])
W2.shape = ([64, 27])
b2.shape = ([27])

```
To find dh and dW2, we'll need to matrix multiply something by dlogits, but what?

For dh, we need the output dimensions to be [32, 64] so, we'll multiply W2 and W2 needs to be transposed so the dimension match up.

So, we get dh = dhpreact @ W2.T

For dW2 we need the output dimensions to be [64, 27], we'll multiply by h and h needs to be transposed

So we get dW2 = h.T @ dhpreact


In [ ]:
# Exercise 2: backprop through cross_entropy but all in one go
# to complete this challenge look at the mathematical expression of the loss,
# take the derivative, simplify the expression, and just write it out

# forward pass

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

3.496117353439331 diff: -2.384185791015625e-07


In [ ]:
# backward pass

# -----------------
# YOUR CODE HERE :)
dlogits = F.softmax(logits, 1)
dlogits[range(n), Yb] -= 1
dlogits /= n
# -----------------

cmp('logits', dlogits, logits) # I can only get approximate to be true, my maxdiff is 6e-9

logits          | exact: False | approximate: True  | maxdiff: 5.587935447692871e-09


In [ ]:
# Exercise 3: backprop through batchnorm but all in one go
# to complete this challenge look at the mathematical expression of the output of batchnorm,
# take the derivative w.r.t. its input, simplify the expression, and just write it out
# BatchNorm paper: https://arxiv.org/abs/1502.03167

# forward pass

# before:
# bnmeani = 1/n*hprebn.sum(0, keepdim=True)
# bndiff = hprebn - bnmeani
# bndiff2 = bndiff**2
# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
# bnvar_inv = (bnvar + 1e-5)**-0.5
# bnraw = bndiff * bnvar_inv
# hpreact = bngain * bnraw + bnbias

# now:
hpreact_fast = bngain * (hprebn - hprebn.mean(0, keepdim=True)) / torch.sqrt(hprebn.var(0, keepdim=True, unbiased=True) + 1e-5) + bnbias
print('max diff:', (hpreact_fast - hpreact).abs().max())

max diff: tensor(4.7684e-07, grad_fn=<MaxBackward1>)


In [ ]:
# backward pass

# before we had:
# dbnraw = bngain * dhpreact
# dbndiff = bnvar_inv * dbnraw
# dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
# dbnvar = (-0.5*(bnvar + 1e-5)**-1.5) * dbnvar_inv
# dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
# dbndiff += (2*bndiff) * dbndiff2
# dhprebn = dbndiff.clone()
# dbnmeani = (-dbndiff).sum(0)
# dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)

# calculate dhprebn given dhpreact (i.e. backprop through the batchnorm)
# (you'll also need to use some of the variables from the forward pass up above)

# -----------------
# YOUR CODE HERE :)
dhprebn = bngain*bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))
# -----------------

cmp('hprebn', dhprebn, hprebn) # I can only get approximate to be true, my maxdiff is 9e-10

hprebn          | exact: False | approximate: True  | maxdiff: 6.984919309616089e-10


In [ ]:
# Exercise 4: putting it all together!
# Train the MLP neural net with your own backward pass

# init
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

# same optimization as last time
max_steps = 200000
batch_size = 32
n = batch_size # convenience
lossi = []

# use this context manager for efficiency once your backward pass is written (TODO)
#with torch.no_grad():

# kick off optimization
for i in range(max_steps):

  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
  Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

  # forward pass
  emb = C[Xb] # embed the characters into vectors
  embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
  # Linear layer
  hprebn = embcat @ W1 + b1 # hidden layer pre-activation
  # BatchNorm layer
  # -------------------------------------------------------------
  bnmean = hprebn.mean(0, keepdim=True)
  bnvar = hprebn.var(0, keepdim=True, unbiased=True)
  bnvar_inv = (bnvar + 1e-5)**-0.5
  bnraw = (hprebn - bnmean) * bnvar_inv
  hpreact = bngain * bnraw + bnbias
  # -------------------------------------------------------------
  # Non-linearity
  h = torch.tanh(hpreact) # hidden layer
  logits = h @ W2 + b2 # output layer
  loss = F.cross_entropy(logits, Yb) # loss function

  # backward pass
  for p in parameters:
    p.grad = None
  # loss.backward() # use this for correctness comparisons, delete it later!

  # manual backprop! #swole_doge_meme
  # -----------------
  # YOUR CODE HERE :)
  dlogits = F.softmax(logits, 1)
  dlogits[range(n), Yb] -= 1
  dlogits /= n
  dW2 = h.T @ dlogits
  dh = dlogits @ W2.T #Full explanation of dh, dW2, b2 below (dimensions need to work out)
  dhpreact = (1.0 - h**2) * dh
  dhprebn = bngain*bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))
  dbngain = (bnraw * dhpreact).sum(0, keepdims=True) #Full exp. of these three below
  dbnraw = bngain * dhpreact
  dbnbias = dhpreact.sum(0, keepdims=True)
  db2 = dlogits.sum(0)
  dembcat = dhprebn @ W1.T
  dW1 = embcat.T @ dhprebn
  db1 = dhprebn.sum(0)
  demb = dembcat.view(emb.shape)
  dC = torch.zeros_like(C)
  for k in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
      ix = Xb[k,j]
      dC[ix] += demb[k,j]

  grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
  # -----------------

  # update
  lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
  for p, grad in zip(parameters, grads):
    # p.data += -lr * p.grad # old way of cheems doge (using PyTorch grad from .backward())
    p.data += -lr * grad # new way of swole doge TODO: enable

  # track stats
  if i % 10000 == 0: # print every once in a while
    print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
  lossi.append(loss.log10().item())

  # if i >= 100: # TODO: delete early breaking when you're ready to train the full net
  #   break

12297
      0/ 200000: 3.5928
  10000/ 200000: 2.4556
  20000/ 200000: 2.3681


KeyboardInterrupt: 

In [ ]:
# useful for checking your gradients
# for p,g in zip(parameters, grads):
#   cmp(str(tuple(p.shape)), g, p)

In [ ]:
# calibrate the batch norm at the end of training

with torch.no_grad():
  # pass the training set through
  emb = C[Xtr]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 + b1
  # measure the mean/std over the entire training set
  bnmean = hpreact.mean(0, keepdim=True)
  bnvar = hpreact.var(0, keepdim=True, unbiased=True)


In [ ]:
# evaluate train and val loss

@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  hpreact = embcat @ W1 + b1
  hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
  h = torch.tanh(hpreact) # (N, n_hidden)
  logits = h @ W2 + b2 # (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

In [ ]:
# I achieved:
# train 2.0718822479248047
# val 2.1162495613098145

In [ ]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):

    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # forward pass
      emb = C[torch.tensor([context])] # (1,block_size,d)
      embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
      hpreact = embcat @ W1 + b1
      hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
      h = torch.tanh(hpreact) # (N, n_hidden)
      logits = h @ W2 + b2 # (N, vocab_size)
      # sample
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break

    print(''.join(itos[i] for i in out))